# Sprint12 深層学習スクラッチ 畳み込みニューラルネットワーク2

## 1.このSprintについて

### Sprintの目的
- スクラッチを通してCNNの基礎を理解する

### どのように学ぶか
スクラッチで2次元用畳み込みニューラルネットワークを実装した後、学習と検証を行なっていきます。

## 2.2次元の畳み込みニューラルネットワークスクラッチ
2次元に対応した畳み込みニューラルネットワーク（CNN）のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

プーリング層なども作成することで、CNNの基本形を完成させます。クラスの名前はScratch2dCNNClassifierとしてください。

### データセットの用意
引き続きMNISTデータセットを使用します。2次元畳み込み層へは、28×28の状態で入力します。

今回は白黒画像ですからチャンネルは1つしかありませんが、チャンネル方向の軸は用意しておく必要があります。

(n_samples, n_channels, height, width)のNCHWまたは(n_samples, height, width, n_channels)のNHWCどちらかの形にしてください。

In [1]:
import numpy as np
import random

from sklearn.metrics import accuracy_score
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
(X_train, t_train), (X_test, t_test) = mnist.load_data()

Using TensorFlow backend.


In [2]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [3]:
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
t_train_one_hot = enc.fit_transform(t_train[:, np.newaxis])
t_test_one_hot = enc.fit_transform(t_test[:,  np.newaxis])

In [4]:
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

In [5]:
class Conv2d:
    
    def __init__(self, initializer, out_chanel, in_chanel, height, width, optimizer):
        init = initializer
        self.w = init.W(out_chanel, in_chanel, height, width)
        self.b = init.B(out_chanel)
        self.optimizer = optimizer
    
    def forward(self, X):
        self.sample_size, self.in_chanel, self.x_height, self.x_width = X.shape
        self.out_chanel, self.inchanel, self.w_height, self.w_width = self.w.shape
        self.XB = X
        
        
        A  = np.zeros([self.sample_size,  self.out_chanel, self.x_height -2, self.x_width -2])
        for n in range(self.sample_size):
            for outchan in range(self.out_chanel):
                for inchan in range(self.in_chanel):
                    for i in range(self.x_height-2):
                        for j in range(self.x_width-2):
                            sig = 0
                            for s in range(self.w_height):
                                for t in range(self.w_width):
                                    sig += X[n, inchan, i+s, j+t] * self.w[outchan, inchan, s, t]
                        A[n, outchan, i, j] += sig + self.b[outchan]
        return A
    
    def backward(self, dA):
        n_out_h , n_out_w = N_out(self.x_height, self.x_width, 0, self.w_height, self.w_width, 1)
        self.lb = dA.sum(axis=(0, 2, 3))
        
        self.lw = np.zeros_like(self.w)
        for n in range(self.sample_size):
            for m in range(self.out_chanel):
                for k in range(self.in_chanel):
                    for s in range(self.w_height):
                        for t in range(self.w_width):
                            for i in range(self.w_height-1):
                                for j in range(self.w_width-1):
                                    self.lw[m, k, s, t] += dA[n, m, i, j] * self.XB[n, k, i+s, j+t]
        
        
        dZ = np.zeros_like(self.XB)
        for n in range(self.sample_size):
            for m in range(self.out_chanel):
                for k in range(self.inchanel):
                    for i in range(self.x_height):
                        for j in range(self.x_width):
                            sig = 0
                            for s in range(self.w_height):
                                for t in range(self.w_width):
                                    if i-s<0 or i-s>n_out_h-1 or j-t < 0 or j-t>n_out_w-1:
                                        pass
                                    else:
                                        sig += dA[n, m, i-s, j-t] * self.w[m, k, s, t]
                            dZ[n, k, i, j] += sig
        
        
        
        self = self.optimizer.update(self)
        return dZ

### 【問題1】2次元畳み込み層の作成
1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。

フォワードプロパゲーションの数式は以下のようになります。
$$a_{i, j, m} = \sum_{k=0}^{K-1} \sum_{s=0}^{F_h -1}\sum_{t=0}^{F_w -1} x_{(i+s),(j+t),k}w_{s,t,k,m} + b_m$$

$a_{i,j,m}$  : 出力される配列のi行j列、mチャンネルの値

$i$ : 配列の行方向のインデックス

$j$ : 配列の列方向のインデックス

$m$ : 出力チャンネルのインデックス

$K$ : 入力チャンネル数

$F_h,F_w$ : 高さ方向（h）と幅方向（w）のフィルタのサイズ

$x_{(i+s),(j+t),k}$ : 入力の配列の(i+s)行(j+t)列、kチャンネルの値

$w_{s,t,k,m}$ : 重みの配列のs行t列目。kチャンネルの入力に対して、mチャンネルへ出力する重み

$b_m$ : mチャンネルへの出力のバイアス項

全てスカラーです。

次に更新式です。1次元畳み込み層や全結合層と同じ形です。
$$w'_{s, t, k, m} = w_{s, t, k, m} - \alpha \frac{\partial L}{\partial w_{s, t, k, m}}$$

$$b'_m = b_m - \alpha \frac{\partial L}{\partial b_m}$$

$\alpha$  : 学習率

$\frac{\partial L}{\partial w_{s, t, k, m}}$ ： $w_{s,t,k,m}$  に関する損失 $L$ の勾配

$\frac{\partial L}{\partial b_m}$ : $b_m$  に関する損失$L$ の勾配

勾配$\frac{\partial L}{\partial w_{s, t, k, m}}$ や $\frac{\partial L}{\partial b_m}$ を求めるためのバックプロパゲーションの数式が以下である。
$$\frac{\partial L}{\partial w_{s, t, k, m}} = \sum_{i=0}^{N_{out, h}-1} \sum_{j=0}^{N_{out, w}-1} \frac{\partial L}{\partial a_{i,j,m}} x_{(i+s)(j+k),k}$$
$$\frac{\partial L}{\partial b_m} =  \sum_{i=0}^{N_{out, h}-1} \sum_{j=0}^{N_{out, w}-1} \frac{\partial L}{\partial a_{i,j,m}}$$

$\frac{\partial L}{\partial a_{i,j,m}}$ : 勾配の配列のi行j列、mチャンネルの値

$N_{out,h},N_{out,w}$  : 高さ方向（h）と幅方向（w）の出力のサイズ

前の層に流す誤差の数式は以下です。
$$\frac{\partial L}{\partial x_{i,j,k}} = \sum_{m=0}^{M-1} \sum_{s=0}^{F_{h-1}} \sum_{t=0}^{F_{w-1}} \frac{\partial L}{\partial a_{(i-s)(j-t),m}}w_{s, t, k, m}$$

$\frac{\partial L}{\partial x_{i,j,k}}$ : 前の層に流す誤差の配列のi列j行、kチャンネルの値
$M$  : 出力チャンネル数 ただし、$i−s<0$ または$i−s>N_{out,h}−1$ または$j−t<0$ または $j−t>N_{out,w}−1$ のとき $\frac{\partial L}{\partial a_{(i-s)(j-t),m}}w_{s, t, k, m}=0$です。

### 【問題2】2次元畳み込み後の出力サイズ
畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。
$$N_{h, out} = \frac{N_{h, in} + 2P_h - F_h}{S_h} + 1$$
$$N_{w, out} = \frac{N_{w, in} + 2P_w - F_w}{S_w} + 1$$

$N_{out}$  : 出力のサイズ（特徴量の数）

$N_{in}$ : 入力のサイズ（特徴量の数）

$P$ : ある方向へのパディングの数

$F$ : フィルタのサイズ

$S$ : ストライドのサイズ

$h$ が高さ方向、$w$ が幅方向である

In [6]:
def N_out(Nh_in, Nw_in, P, Fh, Fw, S):
    Nh_out = ((Nh_in  + 2*P - Fh) / S) + 1
    Nw_out = ((Nw_in + 2*P-Fw) / S) + 1
    return int(Nh_out), int(Nw_out)

### 【問題3】最大プーリング層の作成
最大プーリング層のクラスMaxPool2Dを作成してください。プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。
$$a_{i,j,k} = max_{(p,q)\in P_{i,j}} x_{p,q,k}$$

$P_{i,j}$  : i行j列への出力する場合の入力配列のインデックスの集合。 $S_h×S_w$ の範囲内の行$（p）$と列$（q）$

$S_h,S_w$ : 高さ方向$（h）$と幅方向$（w）$のストライドのサイズ

$(p,q)\in P_{i,j}$ : $P_{i,j}$ に含まれる行$（p）$と列$（q）$のインデックス

$a_{i,j,m}$ : 出力される配列のi行j列、kチャンネルの値

$x_{p,q,k}$ : 入力の配列の$p$行$q$列、$k$チャンネルの値

ある範囲の中でチャンネル方向の軸は残したまま最大値を計算することになります。

バックプロパゲーションのためには、フォワードプロパゲーションのときの最大値のインデックス $(p,q)$ を保持しておく必要があります。フォワード時に最大値を持っていた箇所にそのままの誤差を流し、そこ以外には0を入れるためです。

In [7]:
class MaxPool2D:
    """
    pooling層のクラス
    2*2の正方行列内の最大値をリストに格納
    リストをreshapeすることで行列として返す
    
    2*2の正方行列内の最大値のインデックスを縦横それぞれ別のリストに格納
    要素が0の行列を作り、backwardで帰ってきた値を最大値のインデックスがあった場所へ代入
    """
    
    def forward(self,A):
        self.AB = A
        pooling_list = []
        self.h_list = []
        self.w_list = []
        for n in range(A.shape[0]):
            for cha in range(A.shape[1]):
                for h in range(0, 26, 2):
                    for w in range(0, 26, 2):
                        pooling_list = np.append(pooling_list, np.max(A[n, cha, h:h+2, w:w+2]))
                        self.h_list.append(np.unravel_index(np.argmax(A[n, cha, h:h+2, w:w+2]), A[n, cha, h:h+2, w:w+2].shape)[0])
                        self.w_list.append(np.unravel_index(np.argmax(A[n, cha, h:h+2, w:w+2]), A[n, cha, h:h+2, w:w+2].shape)[1])
                        
        pooing = pooling_list.reshape(A.shape[0], A.shape[1], 13, 13)
        return pooing
    
    def backward(self, dA):
        d = np.zeros_like(self.AB)
        i = 0
        for n in range(dA.shape[0]):
            for cha in range(dA.shape[1]):
                for h in range(0, 26, 2):
                    for w in range(0, 26, 2):
                        d[n, cha, h:h+2, w:w+2][self.h_list[i]][self.w_list[i]] = dA.reshape(-1)[i]
                        i+=1
        return d

### 【問題4】（アドバンス課題）平均プーリングの作成
平均プーリング層のクラスAveragePool2Dを作成してください。

範囲内の最大値ではなく、平均値を出力とするプーリング層です。

画像認識関係では最大プーリング層が一般的で、平均プーリングはあまり使われません。

### 【問題5】平滑化
平滑化するためのFlattenクラスを作成してください。

フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。

この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。

In [8]:
class Flatten:
    def forward(self, X):
        self.X_shape = X.shape
        return X.reshape(X.shape[0], -1)
    
    def bakcward(self, dA):
        return dA.reshape(self.X_shape)

## 3.検証

### 【問題6】学習と推定
作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。

精度は低くともまずは動くことを目指してください。

In [9]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        init = initializer
        self.n_nodes1 = n_nodes1
        self.w = init.W(n_nodes1, n_nodes2)
        self.b = init.B(n_nodes2)
    

    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.z = X
        self.a = X@self.w + self.b
        
        return self.a

    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = dA @ self.w.T
        self.lw = self.z.T @ dA
        self.lb = np.sum(dA, axis=0)
        
        
        # 更新
        self = self.optimizer.update(self)
        return dZ

In [10]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        
        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B  = self.sigma * np.random.randn(n_nodes2)
        return B

In [11]:
class SimpleInitializer_cnn:
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, out_chanel, in_chanel, height, width):

        W = self.sigma * np.random.randn(out_chanel, in_chanel, height, width)
        
        return W
    
    def B(self, out_chanel):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B  = self.sigma * np.random.randn(out_chanel)
        return B

In [12]:
class SGD:

    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        
        layer.w = layer.w -  self.lr * layer.lw
        layer.b = layer.b - self.lr*layer.lb
        
        return layer

In [13]:
class Softmax:
    def forward(self, A):
        exp_a = np.exp(A)
        softmax_result = np.empty((A.shape[0], A.shape[1]))
        exp_sum = np.sum(exp_a, axis=1)
        for i in range(A.shape[0]):
            softmax_result[i] = exp_a[i] / exp_sum[i]
            
        return softmax_result
    
    def backward(self, Z, Y):
        
        L_A = Z - Y
        self.cross_entropy = -np.average(np.sum(Y*np.log(Z), axis=1))
        
        
        return L_A

In [14]:
class Relu:
    def forward(self, X):
        self.A = X
        return np.maximum(0, X)
    
    def backward(self, Z):
        
        return Z * np.maximum(np.sign(self.A), 0)

In [15]:
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

In [16]:
#スクラッチが悪いせいで時間がかかるため、データ数を減らす
X_train_min = X_train[:3000]

t_train_min = t_train_one_hot[:3000]

X_test_min = X_test[:500]

t_test_min = t_test_one_hot[:500]

conv2d = Conv2d(SimpleInitializer_cnn(0.01), 3, 1, 3, 3, SGD(0.1))
maxpool = MaxPool2D()
relu = Relu()
fc1 = FC(507, 10, SimpleInitializer(0.01), SGD(0.1))
softmax = Softmax()

flat = Flatten()

In [17]:
test_A = conv2d.forward(X_test_min)
test_A_2 = relu.forward(test_A)
test_A_3 = maxpool.forward(test_A_2)

test_A_4 = flat.forward(test_A_3)
test_A_5 = fc1.forward(test_A_4)

test_A_6 = softmax.forward(test_A_5)

y = np.argmax(test_A_6 , axis=1)

print(accuracy_score(t_test[:500], y))

0.09


### 【問題7】（アドバンス課題）LeNet
CNNで画像認識を行う際は、フィルタサイズや層の数などを１から考えるのではなく、有名な構造を利用することが一般的です。現在では実用的に使われることはありませんが、歴史的に重要なのは1998年の LeNet です。この構造を再現してMNISTに対して動かし、Accuracyを計算してください。

Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11):2278–2324, 1998.

※上記論文から引用

サブサンプリングとは現在のプーリングに相当するものです。現代風に以下のように作ってみることにします。活性化関数も当時はシグモイド関数ですが、ReLUとします。

畳み込み層　出力チャンネル数6、フィルタサイズ5×5、ストライド1
ReLU
最大プーリング
畳み込み層　出力チャンネル数16、フィルタサイズ5×5、ストライド1
ReLU
最大プーリング
平滑化
全結合層　出力ノード数120
ReLU
全結合層　出力ノード数84
ReLU
全結合層　出力ノード数10
ソフトマックス関数

### 【問題8】（アドバンス課題）有名な画像認識モデルの調査
CNNの代表的な構造としてははAlexNet(2012)、VGG16(2014)などがあります。こういったものはフレームワークで既に用意されていることも多いです。

どういったものがあるか簡単に調べてまとめてください。名前だけでも見ておくと良いでしょう。

《参考》

Applications - Keras Documentation

### 【問題9】出力サイズとパラメータ数の計算
CNNモデルを構築する際には、全結合層に入力する段階で特徴量がいくつになっているかを事前に計算する必要があります。

また、巨大なモデルを扱うようになると、メモリや計算速度の関係でパラメータ数の計算は必須になってきます。フレームワークでは各層のパラメータ数を表示させることが可能ですが、意味を理解していなくては適切な調整が行えません。

以下の3つの畳み込み層の出力サイズとパラメータ数を計算してください。パラメータ数についてはバイアス項も考えてください。

1.

入力サイズ : 144×144, 3チャンネル
フィルタサイズ : 3×3, 6チャンネル
ストライド : 1
パディング : なし
2.

入力サイズ : 60×60, 24チャンネル
フィルタサイズ : 3×3, 48チャンネル
ストライド　: 1
パディング : なし
3.

入力サイズ : 20×20, 10チャンネル
フィルタサイズ: 3×3, 20チャンネル
ストライド : 2
パディング : なし
＊最後の例は丁度良く畳み込みをすることができない場合です。フレームワークでは余ったピクセルを見ないという処理が行われることがあるので、その場合を考えて計算してください。端が欠けてしまうので、こういった設定は好ましくないという例です。

### 【問題10】（アドバンス課題）フィルタサイズに関する調査
畳み込み層にはフィルタサイズというハイパーパラメータがありますが、2次元畳み込み層において現在では3×3と1×1の使用が大半です。以下のそれぞれを調べたり、自分なりに考えて説明してください。

7×7などの大きめのものではなく、3×3のフィルタが一般的に使われる理由
高さや幅方向を持たない1×1のフィルタの効果